In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple

# Handover event function

In [ ]:
def parse_mi_ho(f):

    df = pd.read_csv(f)
    df["Timestamp"] = df["Timestamp"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8)) 
    nr_pci = 'O'
    scells = []

    def NR_OTA(idx):

        if df["type_id"].iloc[idx] == "5G_NR_RRC_OTA_Packet": return True
        else: return False
    
    def LTE_SERV_INFO(idx):

        if df["type_id"].iloc[idx] == "LTE_RRC_Serv_Cell_Info": return True
        else: return False
    

    def find_1st_after(start_idx, target, look_after=1):
        for j in range(start_idx, len(df)):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t_ - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before(start_idx, target, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before_with_special_value(start_idx, target, target_value, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] in [target_value] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_in_D_exact(targets):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.start).total_seconds(), target))

        if len(l) != 0:
            for x in l:
                if (x[0]== 0):
                    return x[1]
        
        return None
    
    def find_in_D_first_before(targets, look_before=1):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.end).total_seconds(), target, ho))

        if len(l) != 0:
            closest = min(filter(lambda x: x[0] > 0, l), key=lambda x: x[0])
            if 0 <= closest[0] < look_before:
                return closest[1], closest[2]
        
        return None, None

    HO = namedtuple('HO',['start', 'end', 'others', 'trans'], defaults=[None,None,'',''])
    
    D = {
        'Conn_Rel':[], 
        'Conn_Req':[], # Setup
        'LTE_HO': [], # LTE -> newLTE
        'MN_HO': [], # LTE + NR -> newLTE + NR
        'MN_HO_to_eNB': [], # LTE + NR -> newLTE
        'SN_setup': [], # LTE -> LTE + NR => NR setup
        'SN_Rel': [], # LTE + NR -> LTE
        'SN_HO': [], # LTE + NR -> LTE + newNR  
        'RLF_II': [],
        'RLF_III': [],
        'SCG_RLF': [],
        'Add_SCell': [],
        }
    
    for i in range(len(df)):

        # Pass NR RRC packet. In NSA mode, LTE RRC packet include NR packet message.
        if NR_OTA(i) or LTE_SERV_INFO(i):
            continue

        try: lte_pci, lte_earfcn
        except: 
            lte_pci = df["PCI"].iloc[i]
            lte_earfcn = int(df["Freq"].iloc[i])

        others = ''
        t = df["Timestamp"].iloc[i]

        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))
            nr_pci = 'O'

        if df["rrcConnectionRequest"].iloc[i] == 1:
            
            # Define end of rrcConnectionRequest to be rrcConnectionReconfigurationComplete or securityModeComplete.
            a = find_1st_after(i, 'rrcConnectionReconfigurationComplete',look_after=2)[0]
            b = find_1st_after(i, 'securityModeComplete',look_after=2)[0]
        
            if a is None and b is None: end = None
            elif a is None and b is not None: end = b
            elif a is not None and b is None: end = a 
            else: end = a if a > b else b
            
            _, idx = find_1st_after(i, 'ueCapabilityInformation',look_after=1)
            if idx is not None:
                sup_band = df['bandEUTRA'].iloc[idx]
                others += f' supported band: {sup_band}.' 

            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            trans = f'({lte_pci}, {lte_earfcn}) -> ({serv_cell}, {serv_freq})'
            
            # Check if caused by RLF III.
            a, idx = find_1st_before(i, 'rrcConnectionReestablishmentReject', look_before=1)
            if a is not None:
                others += ' After RLF III.'

            D['Conn_Req'].append(HO(start=t,end=end,trans=trans, others=others))

            nr_pci = 'O'
            lte_pci = serv_cell
            lte_earfcn = serv_freq
            
        if df["lte-rrc.t304"].iloc[i] == 1:
            
            end, _ = find_1st_after(i, 'rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], int(df['lte_targetPhysCellId'].iloc[i])
            serv_freq, target_freq = int(df["Freq"].iloc[i]), int(df['dl-CarrierFreq'].iloc[i])

            lte_pci = target_cell
            lte_earfcn = target_freq

            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others += f' Set up {n} SCell.'
            else:
                scells = []
            
            if serv_freq != target_freq:
                a,b = find_1st_before(i, "rrcConnectionReestablishmentRequest", 1)
                others += " Inter frequency HO."
                if a is not None:
                    others += " Near after RLF."
                
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 2)
                    
                    if a is not None:

                        ho_type, ho = find_in_D_first_before(['RLF_II', 'RLF_III'], 2)
                        others += f' Near after RLF of trans: {ho.trans}.'

                    else:
                        
                        ho_type, _ = find_in_D_first_before(['MN_HO_to_eNB', 'SN_Rel'], 2)
                        if ho_type is not None:
                            others += f' Near after {ho_type}.'

                    ori_serv = nr_pci
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) | {ori_serv} -> {nr_pci}'
                    D['SN_setup'].append(HO(start=t, end=end, others=others, trans=trans))

                else:
                    
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                    D['MN_HO'].append(HO(start=t, end=end, others=others, trans=trans))

            else:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, b = find_1st_before(i, "scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    
                    orig_serv = nr_pci
                    nr_pci = 'O'
                    trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                    D['SN_Rel'].append(HO(start=t, end=end, others=others, trans=trans))
                    
                else:

                    a, _ = find_1st_before(i,"rrcConnectionSetup",3)
                    if a is not None:
                        others += ' Near After connection setup.'
                    if nr_pci == 'O':
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                        D['LTE_HO'].append(HO(start=t, end=end, others=others, trans=trans))
                    else:
                        orig_serv = nr_pci
                        nr_pci = 'O'
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {orig_serv} -> {nr_pci}'
                        D['MN_HO_to_eNB'].append(HO(start=t, end=end, others=others, trans=trans))


        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
        
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            orig_serv = nr_pci
            nr_pci = int(df['nr_physCellId'].iloc[i])
            trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
            D['SN_HO'].append(HO(start=t,end=end,trans=trans))


        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:

            end1, _ = find_1st_after(i, 'rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after(i, 'rrcConnectionReestablishmentReject', look_after=1)
            end2, _ = find_1st_after(i, 'securityModeComplete',look_after=3)

            others += ' ' + df["reestablishmentCause"].iloc[i] + '.'
            scells = []

            c, _ = find_1st_before(i, 'scgFailureInformationNR-r15', 1)
            if c != None:
                others  += ' caused by scgfailure.'
                
            serv_cell, rlf_cell = df["PCI"].iloc[i], int(df['physCellId.3'].iloc[i])
            serv_freq = int(df['Freq'].iloc[i])
            
            # Type II & Type III
            if end1 is not None: 

                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                rlf_freq = int(df['Freq'].iloc[idx])
                trans = f'({rlf_cell}, {rlf_freq}) -> ({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_II'].append(HO(start=t,end=end1,others=others,trans=trans))

                lte_pci = serv_cell
                lte_earfcn = serv_freq

            elif b is not None and end2 is not None:
                
                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                rlf_freq = int(df['Freq'].iloc[idx])

                _, idx = find_1st_after(i, "rrcConnectionRequest", 2)
                recon_cell, recon_freq = df['PCI'].iloc[idx], int(float(df['Freq'].iloc[idx]))
                
                trans = f'({rlf_cell}, {rlf_freq}) -> ({recon_cell}, {recon_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_III'].append(HO(start=t,end=end2,others=others,trans=trans)) 

                # lte_pci, lte_earfcn will be updated in rrcConnectionRequest.     
                
            else:

                others+=' No end.'
                D['RLF_II'].append(HO(start=t,others=others))
                print('No end for RLF')

        if df["scgFailureInformationNR-r15"].iloc[i] == 1:

            others += ' ' + df["failureType-r15"].iloc[i] + '.'
            a, idx1 = find_1st_after(i, "rrcConnectionReestablishmentRequest", look_after=1)
            b, idx2 = find_1st_after(i, "lte-rrc.t304", look_after=10)

            if a is not None:

                end1, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentComplete', look_after=1)
                b, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentReject', look_after=1)
                end2 = find_1st_after(idx1, 'securityModeComplete',look_after=3)[0]

                others += ' Result in rrcReestablishment.'
                    
                # Type II & Type III Result
                if end1 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end1,others=others))
                elif b is not None and end2 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end2,others=others))
                else:
                    others += ' No end.'
                    D['SCG_RLF'].append(HO(start=t,others=others))
                    print('No end for scg failure result in rrcReestablishment.')

            elif b is not None:

                end, _ = find_1st_after(idx2, 'rrcConnectionReconfigurationComplete')
                serv_cell, target_cell = df["PCI"].iloc[idx2], df['lte_targetPhysCellId'].iloc[idx2]
                serv_freq, target_freq = int(df["Freq"].iloc[idx2]), df['dl-CarrierFreq'].iloc[idx2]
                # We do not change nr_pci here. Instead, we will change it at gNB_Rel event.
                trans = f'({serv_cell}, {serv_freq}) | {nr_pci} -> O'
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    others += ' Result in gNB release.'
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))
                else:
                    others += ' Result in MN HO to eNB.'
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))                  

            else:

                print('No end for scg failure.')
                others += ' No end.'
                D['SCG_RLF'].append(HO(start=t,others=others))
        
        if df['SCellToAddMod-r10'].iloc[i] == 1 and df['physCellId-r10'].iloc[i] != 'nr or cqi report':

            others = ''
            pcis = str(df["physCellId-r10"].iloc[i]).split('@')
            freqs = str(df["dl-CarrierFreq-r10"].iloc[i]).split('@')
            orig_scells = scells
            scells = [(int(float(pci)), int(float(freq))) for pci, freq in zip(pcis, freqs)]

            others += f' Set up {len(scells)} SCell.'
            trans = f'{orig_scells} -> {scells}'

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
            
            a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 3)
            if a is not None:
                others += ' Near after RLF.'

            a = find_in_D_exact(['LTE_HO', 'MN_HO', 'MN_HO_to_eNB', 'SN_setup', 'SN_Rel'])
            if a is not None:
                others += f' With {a}.'

            D['Add_SCell'].append(HO(start=t,end=end,others=others, trans=trans))
    
    return D


# Read file

In [ ]:
## Mobileinsight csv file~
file = '/home/wmnlab/D/database/2023-09-12_1/Modem_Control_Group/qc00/#01/data/diag_log_qc00_2023-09-12_08-29-41_rrc.csv'
mi_rrc_df = pd.read_csv(file)
mi_rrc_df["Timestamp"] = pd.to_datetime(mi_rrc_df["Timestamp"]) + dt.timedelta(hours=8)

file2 = '/home/wmnlab/D/database/2023-09-12_1/Modem_Control_Group/qc00/#01/data/diag_log_qc00_2023-09-12_08-29-41_ml1.csv'
mi_ml1_df = pd.read_csv(file2)
mi_ml1_df["Timestamp"] = pd.to_datetime(mi_ml1_df["Timestamp"]) + dt.timedelta(hours=8)

file3 = '/home/wmnlab/D/database/2023-09-12_1/Modem_Control_Group/qc00/#01/data/diag_log_qc00_2023-09-12_08-29-41_nr_ml1.csv'
mi_nr_ml1_df = pd.read_csv(file3)
mi_nr_ml1_df["Timestamp"] = pd.to_datetime(mi_nr_ml1_df["Timestamp"]) + dt.timedelta(hours=8)

# LTE Signal Strength

In [ ]:
class data_point:
    def __init__(self, pd_data): ## Input pd_df.iloc[index]
        self.time = pd_data['Timestamp']
        self.pci = pd_data['PCI']
        self.servingcell = pd_data['Serving Cell Index']
        self.earfcn = pd_data['EARFCN']
        self.RSRPQ = float(pd_data['RSRP(dBm)']), float(pd_data['RSRQ(dB)'])
        self.neighbor = self.get_nei_pci_signal_strength(pd_data)

    def get_nei_pci_signal_strength(self, pd_data):
        nei_cells = {}
        for i in range(9, len(pd_data), 3):
            if np.isnan(pd_data[i]):
                break
            else:
                nei_cells[int(pd_data[i])] = (float(pd_data[i+1]), float(pd_data[i+2]))                
        return nei_cells

class Signal_Strength_map:
    def __init__(self, mi_ml1_df, mi_rrc_df=None):
        self.L = self.data_list(mi_ml1_df)
        if mi_rrc_df is not None:
            self.ho_event = self.collect_ho_event(mi_rrc_df)
    def data_list(self, mi_ml1_df):
        L = []
        for i in range(len(mi_ml1_df)):
            L.append(data_point(mi_ml1_df.iloc[i]))
        return L

    def draw_pci_rsrp(self, pci_list, x_lim=None, y_lim=None, show_earfcn=False, show_ho=[]): ## Format of l: [('pci1', 'earfcn1'), (pci2, earfcn2), (pci3, earfcn3),...] 
        pci_wanted = [i[0] for i in pci_list]
        earfcn_wanted = [i[1] for i in pci_list]

        if len(pci_list) == 0:
            print("Empty pci list.")
            return
        else:
            x_lists = [] ## Time x = [x1, x2, x3, ...]
            rsrp_lists = [] ## rsrp = [rsrp1, rsrp2, rsrp3, ...]
            for i in range(len(pci_list)):
                x_lists.append([])
                rsrp_lists.append([])

            for i in range(len(self.L)):
                data = self.L[i]
                neighbor_list = list(data.neighbor.keys())
                
                for j in range(len(pci_list)):
                    if data.pci == pci_wanted[j] and data.earfcn == earfcn_wanted[j]:
                        x_lists[j].append(data.time)
                        rsrp_lists[j].append(data.RSRPQ[0])

                    for nei_pci in neighbor_list:
                        if nei_pci == pci_wanted[j] and data.earfcn == earfcn_wanted[j]:
                            x_lists[j].append(data.time)
                            rsrp_lists[j].append(data.neighbor[nei_pci][0])
            
        plt.figure(figsize=(15, 3))
        for i in range(len(x_lists)):
            diff = [(x_lists[i][j+1] - x_lists[i][j]).total_seconds() for j in range(len(x_lists[i])) if j != len(x_lists[i]) -1 ]
            test = 3

            l = []
            for j in range(len(diff)):
                if diff[j] > test: ## If diff > test, no connected
                    l.append(j)
            
            count = 0
            for j in l:
                x_lists[i].insert(j+count+1, x_lists[i][j+count] + dt.timedelta(seconds=test))
                rsrp_lists[i].insert(j+count+1, np.nan)
                count += 1
            
            plt.plot(x_lists[i],rsrp_lists[i])

        axes = plt.gca()
        y_min, y_max = axes.get_ylim()
        x_min, x_max = axes.get_xlim()

        colors = ['red', 'green', 'pink', 'cyan', 'lightskyblue']
        if len(show_ho) != 0:
            d = self.ho_event
            i = 0
            for ho in show_ho:
                a = [ i for i in self.ho_event[ho]]
                plt.vlines(a,ymin=y_min, ymax=y_max,label=ho, color=colors[i])
                i += 1

        if show_earfcn:
            legend = [f'{p} ({e})' for p,e in zip(pci_wanted, earfcn_wanted)] 
            plt.legend(legend+show_ho)
        else:
            plt.legend(pci_wanted+show_ho)
            
        if x_lim is not None:
            plt.xlim(x_lim)
        else:
            plt.xlim(x_min, x_max)
        
        if y_lim is not None:
            plt.ylim(y_lim)
        # plt.gcf().autofmt_xdate()
        plt.ylabel("RSRP")
        plt.xlabel("Time")
        plt.show()

    def draw_ho_event(self, ho_type, x_lim=None):
        fig = plt.figure(figsize=(15, 3))
        colors = ['red', 'green', 'pink', 'cyan', 'lightskyblue']
        i = 0
        for type in ho_type:
            if type in list(self.ho_event.keys()):
                a = self.ho_event[type]
                plt.vlines(a,ymin=0, ymax=1,label=type, color=colors[i])
                i += 1
        
        if x_lim is not None:
            plt.xlim(x_lim)
            
        plt.legend()
        plt.show()

In [ ]:
x = Signal_Strength_map(mi_ml1_df, mi_rrc_df)

In [ ]:
x.ho_event

In [ ]:
pci_list = [(394, 3050), (178, 3050)]
time_range = [dt.datetime(2023, 2, 4, 16, 55, 0), dt.datetime(2023, 2, 4, 16, 56, 0)]
# time_range = None
show_ho = ['LTE_HO', 'MN_HO']
x.draw_pci_rsrp(pci_list, show_ho=show_ho, x_lim=time_range, show_earfcn=True)

In [ ]:
pci_list = [('79', '9560'), ('183','9560'), ('56', '9560'), ('138', '9560'), ('16', '9560'), ('163','9560')]
time_range = [dt.datetime(2022, 9, 29, 16, 25, 0), dt.datetime(2022, 9, 29, 16, 30, 0)]
# time_range = None
# rsrp_range = [-120, -80]
rsrp_range = None
show_ho = ['lte', 'MN_changed', "type3 fail"]
# show_ho = ['SCell_add_rel']

x.draw_pci_rsrp(pci_list, show_earfcn=True, x_lim=time_range, y_lim=rsrp_range, show_ho=show_ho)

# NR Signal Stremgth

In [ ]:
class nr_data_point:
    def __init__(self, pd_data): ## Input pd_df.iloc[index]
        self.time = pd_data['time']
        # self.servingcell_index = pd_data['Serving Cell Index']
        # self.servingcell_index = pd_data['Serving Cell PCI']
        # self.earfcn = pd_data['Raster ARFCN']
        self.neighbor = self.get_nei_pci_signal_strength(pd_data)

    def get_nei_pci_signal_strength(self, pd_data):
        nei_cells = {}
        for i in range(6, len(pd_data), 3):
            if np.isnan(pd_data[i]):
                break
            else:
                if i == 6:
                    nei_cells[pd_data[i]] = (float(pd_data[i+1]), float(pd_data[i+2]))
                else:
                    nei_cells[int(pd_data[i])] = (float(pd_data[i+1]), float(pd_data[i+2]))                
        return nei_cells

class Signal_Strength_map:
    def __init__(self, mi_nr_ml1_df, mi_rrc_df=None):
        self.L = self.data_list(mi_nr_ml1_df)
        if mi_rrc_df is not None:
            self.ho_event = self.collect_ho_event(mi_rrc_df)
    def data_list(self, mi_nr_ml1_df):
        L = []
        for i in range(len(mi_nr_ml1_df)):
            L.append(nr_data_point(mi_nr_ml1_df.iloc[i]))
        return L

    def draw_pci_rsrp(self, pci_list, x_lim=None, y_lim=None, show_ho=[]): ## Format of pci_list: ['pci1', 'pci2', ...] 
        if len(pci_list) == 0:
            print("Empty pci list.")
        else:
            x_lists = [] ## Time x = [x1, x2, x3, ...]
            rsrp_lists = [] ## rsrp = [rsrp1, rsrp2, rsrp3, ...]
            for i in range(len(pci_list)):
                x_lists.append([])
                rsrp_lists.append([])

            for i in range(len(self.L)):
                data = self.L[i]
                neighbor_list = list(data.neighbor.keys())
                
                for j in range(len(pci_list)):
                    for nei_pci in neighbor_list:
                        if nei_pci == pci_list[j]:
                            x_lists[j].append(data.time)
                            rsrp_lists[j].append(data.neighbor[nei_pci][0])
            
        fig = plt.figure(figsize=(15, 3))
        for i in range(len(x_lists)):
            diff = [(x_lists[i][j+1] - x_lists[i][j]).total_seconds() for j in range(len(x_lists[i])) if j != len(x_lists[i]) -1 ]
            test = 3

            l = []
            for j in range(len(diff)):
                if diff[j] > test: ## If diff > test, no connected
                    l.append(j)
            
            count = 0
            for j in l:
                x_lists[i].insert(j+count+1, x_lists[i][j+count] + dt.timedelta(seconds=test))
                rsrp_lists[i].insert(j+count+1, np.nan)
                count += 1
            
            plt.plot(x_lists[i],rsrp_lists[i])

        axes = plt.gca()
        y_min, y_max = axes.get_ylim()
        x_min, x_max = axes.get_xlim()

        colors = ['red', 'green', 'pink', 'cyan', 'lightskyblue']
        if len(show_ho) != 0:
            d = self.ho_event
            i = 0
            for ho in show_ho:
                a = [ i for i in self.ho_event[ho]]
                plt.vlines(a,ymin=y_min, ymax=y_max,label=ho, color=colors[i])
                i += 1


        plt.legend(pci_list+show_ho)
            
        if x_lim is not None:
            plt.xlim(x_lim)
        else:
            plt.xlim(x_min, x_max)
        
        if y_lim is not None:
            plt.ylim(y_lim)
        plt.ylabel("RSRP")
        plt.xlabel("Time")
        plt.show()

    def collect_ho_event(self, mi_rrc_df):
        d = parse_mi_ho(mi_rrc_df)
        for key in d:
            d[key] = [x.start for x in d[key]]
        return d
    
    def draw_ho_event(self, ho_type, x_lim=None):
        fig = plt.figure(figsize=(15, 3))
        colors = ['red', 'green', 'orange', 'cyan', 'lightskyblue']
        i = 0
        for type in ho_type:
            if type in list(self.ho_event.keys()):
                a = self.ho_event[type]
                plt.vlines(a,ymin=0, ymax=1,label=type, color=colors[i])
                i += 1
        
        if x_lim is not None:
            plt.xlim(x_lim)
            
        plt.legend()
        plt.show()
    
    

In [ ]:
x = Signal_Strength_map(mi_nr_ml1_df, mi_rrc_df)

In [ ]:
# x.ho_event

In [ ]:
ho_type = ['SN_setup']
time_range = None
pci_list = [13, 21, 5]
x.draw_pci_rsrp(pci_list=pci_list, x_lim=time_range, show_ho=ho_type)